In [28]:
import json
import numpy as np
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

In [29]:
DATAPATH = "data.json"
SAVED_MODEL_PATH = "model.h5"
LEARNING_RATE = 0.0001
EPOCHS = 40
BATCH_SIZE = 32
NUM_KEYWORDS = 10

In [30]:
def load_dataset(datapath):
    
    with open(datapath, "r") as fp:
        data = json.load(fp)
        
    X = np.array(data["MFCCs"])
    y = np.array(data["labels"])
    print("Training data loaded!")
    return X, y

def get_data_splits(datapath, test_size=0.1, validation_size=0.2):
    
    #load dataset
    X, y = load_dataset(datapath)
    
    #create train/validation/test splits
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train,test_size= validation_size)
    
    # convert inputs from 2d to 3d using nd array  #(num_segments, 13) -> (num_segments,13,1)
    X_train = X_train[..., np.newaxis]
    X_test = X_test[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    
    return X_train, y_train, X_validation, y_validation,  X_test, y_test

In [34]:
def build_model(input_shape, learning_rate=0.0001, error="sparse_categorical_crossentropy"):
    
    #build network
    model = keras.Sequential()
    
    #1st conv layer
    model.add(keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=input_shape,
                                  kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'))
    
    #2nd conv layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu',
                                  kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'))
    
    #3rd conv layer
    model.add(keras.layers.Conv2D(32, (2,2), activation='relu',
                                  kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2), padding='same'))
    
    #flatten output and fed into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    
    #softmax output layer
    model.add(keras.layers.Dense(10, activation='softmax')) #[0.1, 0.7, 0.2]
    
    
    #compile the model
    optimiser = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimiser, loss=error, metrics="accuracy")
    
    # print model
    model.summary()
    
    return model

In [37]:
 def main():
        
        #load train/validation data splits
        X_train, y_train, X_validation, y_validation, X_test, y_test = get_data_splits(DATAPATH)
        
        # build the CNN model
        input_shape = (X_train.shape[1], X_train.shape[2], 1)  #(num_segments, num_coeff, 1)
        model = build_model(input_shape, LEARNING_RATE)
        
        # train the model
        model.fit(X_train, y_train, epochs= EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_validation, y_validation))
        
        #evaluate the model
        test_error, test_accuracy = model.evaluate(X_test, y_test)
        print(f"\nTest Error:{test_error}, Test Accuracy:{test_accuracy}")
        
        # save the model 
        model.save(SAVED_MODEL_PATH)

In [38]:
if __name__ == '__main__':
    main()

Training data loaded!
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 42, 11, 64)        640       
_________________________________________________________________
batch_normalization_21 (Batc (None, 42, 11, 64)        256       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 21, 6, 64)         0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 19, 4, 32)         18464     
_________________________________________________________________
batch_normalization_22 (Batc (None, 19, 4, 32)         128       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 10, 2, 32)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (No